# Register Dataset

In this notebook we register the dataset.

In [1]:
from azureml.core import Workspace, Dataset, Datastore, VERSION
from azureml.core.compute import AmlCompute

In [2]:
VERSION

'1.0.48'

In [3]:
ws = Workspace.from_config()
dstor = ws.datastores['datalake']

In [4]:
ds_train = Dataset.auto_read_files(dstor.path("airlift/data-latest.csv"))
ds_score = Dataset.auto_read_files(dstor.path("airlift/data-to-score.csv"))
ds_train.head(3)

,ACCOUNT_NUMBER,AGE,APP_DOWNLOADED,APP_OFFER_USER,BOUGHT_CATEGORY_10,BOUGHT_CATEGORY_3,BOUGHT_CATEGORY_4,BOUGHT_CATEGORY_5,BOUGHT_CATEGORY_6,BOUGHT_CATEGORY_7,...,COUNTRY_CODE,EMAILABLE,GENDER,HEALTH_CLUB,LATITUDE,LONGITUDE,MAILABLE,ONLINE_SHOPPER,PARENT_CLUB,TENURE
0,48522366,42.0,False,False,N,N,U,Y,U,N,...,ENG,False,M,False,53.792526,1.550590,True,True,False,13
1,80774225,49.0,False,False,N,Y,U,N,N,U,...,ENG,True,M,False,51.526898,0.127894,False,False,False,9
2,50718888,28.0,True,False,U,N,Y,N,N,N,...,ENG,True,M,False,51.525729,0.127879,False,True,True,5


In [5]:
ds_train_def = ds_train.get_definition()
ds_score_def = ds_score.get_definition()

In [6]:
ds_train_def = ds_train_def.rename_columns({"BOUGHT_CATEGORY_FNN":"BOUGHT_CATEGORY_1",
                                "BOUGHT_CATEGORY_WLN":"BOUGHT_CATEGORY_2",
                               "CATEGORY_FNN_CLICKS":"CATEGORY_1_CLICKS",
                               "CATEGORY_WLN_CLICKS":"CATEGORY_2_CLICKS",
                               "CATEGORY_FNN_SPEND":"CATEGORY_1_SPEND",
                               "CATEGORY_WLN_SPEND":"CATEGORY_2_SPEND",
                                "CATEGORY_FNN_COUNT":"CATEGORY_1_COUNT",
                               "CATEGORY_WLN_COUNT":"CATEGORY_2_COUNT"})

ds_score_def = ds_score_def.rename_columns({"CATEGORY_FNN_CLICKS":"CATEGORY_1_CLICKS",
                               "CATEGORY_WLN_CLICKS":"CATEGORY_2_CLICKS",
                               "CATEGORY_FNN_SPEND":"CATEGORY_1_SPEND",
                               "CATEGORY_WLN_SPEND":"CATEGORY_2_SPEND",
                                "CATEGORY_FNN_COUNT":"CATEGORY_1_COUNT",
                               "CATEGORY_WLN_COUNT":"CATEGORY_2_COUNT"})

ds_train_def.head(3).columns

Index(['ACCOUNT_NUMBER', 'AGE', 'APP_DOWNLOADED', 'APP_OFFER_USER',
       'BOUGHT_CATEGORY_10', 'BOUGHT_CATEGORY_3', 'BOUGHT_CATEGORY_4',
       'BOUGHT_CATEGORY_5', 'BOUGHT_CATEGORY_6', 'BOUGHT_CATEGORY_7',
       'BOUGHT_CATEGORY_8', 'BOUGHT_CATEGORY_9', 'BOUGHT_CATEGORY_1',
       'BOUGHT_CATEGORY_2', 'CATEGORY_10_CLICKS', 'CATEGORY_10_COUNT',
       'CATEGORY_10_SPEND', 'CATEGORY_3_CLICKS', 'CATEGORY_3_COUNT',
       'CATEGORY_3_SPEND', 'CATEGORY_4_CLICKS', 'CATEGORY_4_COUNT',
       'CATEGORY_4_SPEND', 'CATEGORY_5_CLICKS', 'CATEGORY_5_COUNT',
       'CATEGORY_5_SPEND', 'CATEGORY_6_CLICKS', 'CATEGORY_6_COUNT',
       'CATEGORY_6_SPEND', 'CATEGORY_7_CLICKS', 'CATEGORY_7_COUNT',
       'CATEGORY_7_SPEND', 'CATEGORY_8_CLICKS', 'CATEGORY_8_COUNT',
       'CATEGORY_8_SPEND', 'CATEGORY_9_CLICKS', 'CATEGORY_9_COUNT',
       'CATEGORY_9_SPEND', 'CATEGORY_1_CLICKS', 'CATEGORY_1_COUNT',
       'CATEGORY_1_SPEND', 'CATEGORY_2_CLICKS', 'CATEGORY_2_COUNT',
       'CATEGORY_2_SPEND', 'COUNTRY_C

In [7]:
ds_train.update_definition(ds_train_def, "column changes")
ds_score.update_definition(ds_score_def, "column changes")
ds_train.register(ws, name="AirliftDataset-Train", exist_ok=True)
ds_score.register(ws, name="AirliftDataset-Score", exist_ok=True)

Dataset(Name: AirliftDataset-Score,
Workspace: dev)

In [8]:
ct = AmlCompute(ws, "onenode-cpu")
ds_train.generate_profile(ct, workspace=ws)
ds_score.generate_profile(ct, workspace=ws)